# XGB Optimization

Experiment with XGB parameter optimization to see if it can be used with my Abalone experiments.

https://medium.com/@attud_bidirt/automatic-tuning-of-hyper-parameters-of-a-xgboost-classifier-c5588bceda4

Note that the code in this example does not work in the order specified.  The code blocks are misarranged, and there is some missing work.  But still interesting!

## Get data right

In [1]:
from sklearn import datasets

# without the as frame this is loaded as Bunch
data = datasets.load_breast_cancer(as_frame=True)

In [15]:
type(data)

sklearn.utils._bunch.Bunch

In [16]:
data.frame

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


In [17]:
data.frame.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension',
       'target'],
      dtype='object')

In [18]:
data.frame.target

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

In [2]:
# Need to separate data into X and Y
X = data.frame.drop('target', axis = 1)
y = data.frame.target

In [20]:
X

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [23]:
y

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Name: target, Length: 569, dtype: int64

In [3]:
from pandas.core.common import random_state
from sklearn.model_selection import StratifiedShuffleSplit

ss = StratifiedShuffleSplit(2,test_size=0.2, random_state=44)
for tr_idx,ts_idx in ss.split(X,y):
  X_train, y_train = X.loc[tr_idx], y.loc[tr_idx]
  X_test, y_test = X.loc[ts_idx], y.loc[ts_idx]

print(f"\nShape of X_train is {X_train.shape}")
print(f"\nShape of X_test is {X_test.shape}")
print(f"\nLength of y_train is {y_train.shape}")
print(f"\nLength of y_test is {y_test.shape}")


Shape of X_train is (455, 30)

Shape of X_test is (114, 30)

Length of y_train is (455,)

Length of y_test is (114,)


## Preliminary Tests

In [26]:
import xgboost as xgb 
from sklearn.model_selection import cross_val_score 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval 
from hyperopt.early_stop import no_progress_loss

In [29]:
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical = True)

In [30]:
from sklearn.metrics import f1_score, recall_score, confusion_matrix,roc_auc_score

params_1 = {"objective": "binary:logistic"}

n = 1000

results = xgb.cv(params_1,
                 dtrain_clf,
                 num_boost_round = n,
                 nfold=5,
                 metrics = ["logloss","auc","error"],
                 early_stopping_rounds=20
                 )

In [31]:
clf_1 = xgb.XGBClassifier(**params_1)

clf_1.fit(X_train,y_train)

pred_1 = clf_1.predict(X_test)

print(f"f1 score : {f1_score(y_test, pred_1)}\n")
print(f"confusion Matrix:\n{confusion_matrix(y_test, pred_1)}\n")

f1 score : 0.9403973509933775

confusion Matrix:
[[34  8]
 [ 1 71]]



## Now Hyperopt

In [34]:
import xgboost as xgb 
from sklearn.model_selection import cross_val_score 
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval 
from hyperopt.early_stop import no_progress_loss
import numpy as np

In [35]:
search_space = {
    'max_depth': hp.choice("max_depth", np.arange(1,20,1,dtype=int)),
    'eta'      : hp.uniform("eta", 0, 1),
    'gamma'    : hp.uniform("gamma", 0, 10e1),
    'reg_alpha': hp.uniform("reg_alpha", 10e-7, 10),
    'reg_lambda' : hp.uniform("reg_lambda", 0,1),
    'colsample_bytree': hp.uniform("colsample_bytree", 0.5,1),
    'colsample_bynode': hp.uniform("colsample_bynode", 0.5,1), 
    'colsample_bylevel': hp.uniform("colsample_bylevel", 0.5,1),
    'n_estimators': hp.choice("n_estimators", np.arange(100,1000,10,dtype='int')),
    'min_child_weight' : hp.choice("min_child_weight", np.arange(1,10,1,dtype='int')),
    'max_delta_step' : hp.choice("max_delta_step", np.arange(1,10,1,dtype='int')),
    'subsample' : hp.uniform("subsample",0.5,1),
    'objective' : 'binary:logistic',
    'eval_metric' : 'aucpr',
    'seed' : 44
}

In [36]:
def xgb_objective(space):
  results = xgb.cv(space, 
                   dtrain=dtrain_clf, #DMatrix (xgboost specific)
                   num_boost_round=500, 
                   nfold=5, 
                   stratified=True,  
                   early_stopping_rounds=20,
                   metrics = ['logloss','auc','aucpr','error'])
  
  best_score = results['test-auc-mean'].max()
  return {'loss':-best_score, 'status': STATUS_OK}

In [37]:
trials = Trials()

best_hyperparams = fmin(fn=xgb_objective, space=search_space,algo=tpe.suggest,max_evals=500,trials=trials, return_argmin=False, early_stop_fn=no_progress_loss(10))

best_params = best_hyperparams.copy()

# `eval_metric` is a key that is not a hyperparameter of the classifier
if 'eval_metric' in best_params:
  best_params = {key:best_params[key] for key in best_params if key!='eval_metric'}

best_params

  0%|          | 1/500 [00:00<00:54,  9.16trial/s, best loss: -0.8919504643962849]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  1%|          | 3/500 [00:00<01:02,  7.93trial/s, best loss: -0.9722910216718266]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  1%|          | 5/500 [00:00<01:08,  7.26trial/s, best loss: -0.972703818369453] 

...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:27] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:27] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▍         | 23/500 [00:04<01:39,  4.81trial/s, best loss: -0.9919504643962849]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:27] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:48:27] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  5%|▍         | 24/500 [00:04<01:32,  5.13trial/s, best loss: -0.9919504643962849]


{'colsample_bylevel': 0.7370830882900052,
 'colsample_bynode': 0.9456419018900569,
 'colsample_bytree': 0.6886367856734236,
 'eta': 0.5586879476999288,
 'gamma': 1.9552221486075605,
 'max_delta_step': 4,
 'max_depth': 9,
 'min_child_weight': 5,
 'n_estimators': 630,
 'objective': 'binary:logistic',
 'reg_alpha': 0.939894791803691,
 'reg_lambda': 0.5277342478394749,
 'seed': 44,
 'subsample': 0.5568365639214825}

## Test with optimized params

Results show that the 'best' params are meh. It may well have overfit

In [38]:
clf_2 = xgb.XGBClassifier(**best_params)
clf_2.fit(X_train, y_train)

pred_2 = clf_2.predict(X_test)

print(f"f1 score : {f1_score(y_test, pred_2)}\n")

f1 score : 0.9459459459459458



In [42]:
search_space_3 = {
    
    'eta'      : hp.uniform("eta", 0, 1),
    'gamma'    : 0,#hp.uniform("gamma", 0, 1),
    'reg_lambda' : hp.uniform("reg_lambda", 0,1),
    'n_estimators': hp.choice("n_estimators", np.arange(100,1000,10,dtype='int')),
    'objective' : 'binary:logistic',
    'eval_metric' : 'auc',
    'seed' : 44
}

Hand tuned search space...

In [45]:
trials = Trials()

best_hyperparams = fmin(fn=xgb_objective, space=search_space_3,algo=tpe.suggest,max_evals=500,trials=trials, return_argmin=False, early_stop_fn=no_progress_loss(10))

best_params = best_hyperparams.copy()

# `eval_metric` is a key that is not a hyperparameter of the classifier
if 'eval_metric' in best_params:
  best_params = {key:best_params[key] for key in best_params if key!='eval_metric'}

best_params

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:53:02] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:53:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  0%|          | 1/500 [00:00<01:57,  4.24trial/s, best loss: -0.9851909184726523]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:53:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  1%|          | 3/500 [00:01<02:50,  2.91trial/s, best loss: -0.9876160990712075]

...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:53:07] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  4%|▍         | 19/500 [00:06<05:09,  1.56trial/s, best loss: -0.9902476780185758]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [08:53:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)



  4%|▍         | 20/500 [00:06<02:43,  2.93trial/s, best loss: -0.9902476780185758]


{'eta': 0.6783655045838408,
 'gamma': 0,
 'n_estimators': 890,
 'objective': 'binary:logistic',
 'reg_lambda': 0.3896554989724714,
 'seed': 44}

In [46]:
clf_2 = xgb.XGBClassifier(**best_params)
clf_2.fit(X_train, y_train)

pred_2 = clf_2.predict(X_test)

print(f"f1 score : {f1_score(y_test, pred_2)}\n")

f1 score : 0.953020134228188



## TODO

This article is interesting buy almost disproves itselt, as the search for better params didn't lead to much.  I should play with hyperopt a bit more to see if I can figure out how to get it to add value.

## Take 2

https://medium.com/@justin.wesley.johns/precision-ml-engineering-with-xgboost-hyperopt-attaining-98-11-accuracy-on-mnist-d737b7ef1081



In [4]:
import xgboost as xgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.datasets import mnist
import numpy as np

In [5]:
# Define the hyperparameter space
space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.1),
    'reg_lambda': hp.uniform('reg_lambda', 0.5, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 600, 50),
    'learning_rate': hp.uniform('learning_rate', 0.05, 0.2),
    'tree_method': 'gpu_hist'  # Use GPU
}


In [12]:

# Define the objective function
def objective(space):
    clf = xgb.XGBClassifier(
        #n_estimators=int(space['n_estimators']),
        #max_depth=int(space['max_depth']),
        gamma=space['gamma'],
        reg_alpha=space['reg_alpha'],
        reg_lambda=space['reg_lambda'],
        colsample_bytree=space['colsample_bytree'],
        #subsample=space['subsample'],
        #learning_rate=space['learning_rate'],
        #tree_method=space['tree_method'],
        #use_label_encoder=False,  # Avoid the deprecation warning
        #eval_metric='mlogloss'
    )
    evaluation = [(X_train, y_train), (X_test, y_test)]
        
    clf.fit(X_train, y_train,
                eval_set=evaluation, verbose=True)
        
    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print(f"SCORE: {accuracy}")

    return {'loss': -accuracy, 'status': STATUS_OK}

In [ ]:
# Run the optimization
trials = Trials()
best_hyperparams = fmin(fn=objective,
                        space=space,
                        algo=tpe.suggest,
                        max_evals=200,
                        trials=trials)
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

[0]	validation_0-logloss:0.43922	validation_1-logloss:0.45901
[1]	validation_0-logloss:0.31471	validation_1-logloss:0.35889
[2]	validation_0-logloss:0.23447	validation_1-logloss:0.29498
[3]	validation_0-logloss:0.17822	validation_1-logloss:0.25126
[4]	validation_0-logloss:0.13910	validation_1-logloss:0.22440
[5]	validation_0-logloss:0.10948	validation_1-logloss:0.20425
[6]	validation_0-logloss:0.08750	validation_1-logloss:0.19383
[7]	validation_0-logloss:0.07265	validation_1-logloss:0.18639
[8]	validation_0-logloss:0.06119	validation_1-logloss:0.18305
[9]	validation_0-logloss:0.05192	validation_1-logloss:0.18090
[10]	validation_0-logloss:0.04525	validation_1-logloss:0.17862
[11]	validation_0-logloss:0.03948	validation_1-logloss:0.17479
[12]	validation_0-logloss:0.03477	validation_1-logloss:0.17059
[13]	validation_0-logloss:0.03129	validation_1-logloss:0.16902
[14]	validation_0-logloss:0.02811	validation_1-logloss:0.16583
[15]	validation_0-logloss:0.02580	validation_1-logloss:0.16805
[1